<a href="https://colab.research.google.com/github/ifeLight/ml-bot/blob/main/binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [70]:

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

binance_base_url = 'https://52on3577u3.execute-api.eu-central-1.amazonaws.com'

def get_binance_candles(symbol: str, interval='1h', limit: int = 50, **kwargs):
    url = f'{binance_base_url}/api/v3/uiKlines?symbol={symbol}&interval={interval}&limit={limit}'
    for key, value in kwargs.items():
        url += f'&{key}={value}'
    response = requests.get(url)
    result = json.loads(response.text)
    # print(result)
    def map_result(x):
        return {
            'Date': x[0],
            'Open': x[1],
            'High': x[2],
            'Low': x[3],
            'Close': x[4],
            'Volume': x[5],
        }
    mappeded_result = []
    for x in result:
        mappeded_result.append(map_result(x))
    return mappeded_result


def candles_to_df(data):
    df =  pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'], unit='ms')
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    return df

def get_all_binance_candles(symbol: str, interval='1h', start_date=None, end_date=None, limit=1000):
  result = []
  start_date = pd.to_datetime(start_date) if start_date else pd.to_datetime('2015-01-01')
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')
  while True:
    candles = get_binance_candles(symbol, interval, limit, startTime=int(start_date.timestamp() * 1000), endTime=int(end_date.timestamp() * 1000))
    if len(candles) <= 1:
      break;
    result += candles
    start_date = pd.to_datetime(datetime.datetime.fromtimestamp(candles[-1]['Date'] / 1000))
  return candles_to_df(result)

In [76]:
def calculate_pivot_probabilities(df, window=5):
    """
    Calculate the probability of price going up and down based on higher and lower pivots.
    """
    df['Higher_Pivot'] = df['High'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: x.iloc[window] == x.max(), raw=False
    )
    df['Lower_Pivot'] = df['Low'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: x.iloc[window] == x.min(), raw=False
    )
    df['Midpoint'] = (df['High'] + df['Low']) / 2

    df['Up_Probability'] = np.nan
    df['Down_Probability'] = np.nan

    for i in range(len(df)):
        if df.loc[df.index[i], 'Lower_Pivot']:
            df.loc[df.index[i], 'Up_Probability'] = 1.0
            df.loc[df.index[i], 'Down_Probability'] = 0.0
        elif df.loc[df.index[i], 'Higher_Pivot']:
            df.loc[df.index[i], 'Up_Probability'] = 0.0
            df.loc[df.index[i], 'Down_Probability'] = 1.0
        else:
            lower_pivot = df['Low'].iloc[max(0, i - window):i + window + 1].min()
            higher_pivot = df['High'].iloc[max(0, i - window):i + window + 1].max()
            midpoint = (higher_pivot + lower_pivot) / 2

            if df.loc[df.index[i], 'Close'] < midpoint:
                up_prob = 1.0 - (df.loc[df.index[i], 'Close'] - lower_pivot) / (midpoint - lower_pivot) * 0.5
                down_prob = 1.0 - up_prob
            else:
                up_prob = 0.5 - (df.loc[df.index[i], 'Close'] - midpoint) / (higher_pivot - midpoint) * 0.5
                down_prob = 1.0 - up_prob

            df.loc[df.index[i], 'Up_Probability'] = up_prob
            df.loc[df.index[i], 'Down_Probability'] = down_prob

    return df

In [71]:
data = get_all_binance_candles('BTCUSDT', '4h', '01-01-2024', '31-12-2024')

<ipython-input-70-1a7def5fc8f4>:41: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')


In [81]:
calculate_pivot_probabilities(data)

,Date,Open,High,Low,Close,Volume,Higher_Pivot,Lower_Pivot,Midpoint,Up_Probability,Down_Probability
0,2024-01-01 00:00:00,42283.58,42775.00,42230.08,42330.49,3948.08335,NaN,NaN,42502.540,1.0,0.0
1,2024-01-01 04:00:00,42330.50,42500.00,42180.77,42492.46,2706.99880,NaN,NaN,42340.385,1.0,0.0
2,2024-01-01 08:00:00,42492.46,42762.39,42452.58,42690.20,2948.80274,NaN,NaN,42607.485,1.0,0.0
3,2024-01-01 12:00:00,42690.21,42847.07,42580.00,42783.05,2936.44406,NaN,NaN,42713.535,1.0,0.0
4,2024-01-01 16:00:00,42783.05,43550.00,42664.42,43517.99,5686.97164,NaN,NaN,43107.210,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2188,2024-12-30 08:00:00,93731.66,94068.56,93469.56,93923.14,1683.90269,NaN,NaN,93769.060,1.0,0.0
2189,2024-12-30 12:00:00,93923.14,94277.23,91530.45,92130.21,9295.96250,NaN,NaN,92903.840,1.0,0.0
2190,2024-12-30 16:00:00,92130.21,94684.09,91800.00,94417.04,6130.98017,NaN,NaN,93242.045,1.0,0.0
2191,2024-12-30 20:00:00,94417.04,95024.50,91901.57,92792.05,5169.96449,NaN,NaN,93463.035,1.0,0.0


In [85]:
data.iloc[10:16]

,Date,Open,High,Low,Close,Volume,Higher_Pivot,Lower_Pivot,Midpoint,Up_Probability,Down_Probability
10,2024-01-02 16:00:00,45184.83,45399.84,44790.96,45039.49,9745.60445,0.0,0.0,45095.400,0.163782,0.836218
11,2024-01-02 20:00:00,45039.49,45297.26,44600.00,44946.91,6655.56882,0.0,0.0,44948.630,0.181830,0.818170
12,2024-01-03 00:00:00,44946.91,45410.91,44862.11,45262.69,4750.68596,0.0,0.0,45136.510,0.120270,0.879730
13,2024-01-03 04:00:00,45262.70,45384.84,44940.93,45011.77,4691.11517,0.0,0.0,45162.885,0.169186,0.830814
14,2024-01-03 08:00:00,45011.77,45500.00,43500.00,43702.88,13237.52778,0.0,0.0,44500.000,0.415271,0.584729
15,2024-01-03 12:00:00,43702.89,43716.00,40750.00,43000.01,35484.15210,0.0,1.0,42233.000,1.000000,0.000000
